In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 12

In [2]:
# 실시간 랭킹 300위 페이지 띄우기
targetSite = 'https://page.kakao.com/menu/10011/screen/94'
driver = webdriver.Chrome() 
driver.get(targetSite)
time.sleep(1)
request = requests.get(targetSite)
html = request.text
soup = BeautifulSoup(html, 'html.parser') # 크롤링 하려는 페이지 띄우기

for i in range(6): # 가장 마지막 까지 스크롤
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(1)

# top300의장르, 제목, 작가 저장할 리스트
Kakao_Genre = [] # 장르 저장할 리스트
Kakao_Titles = [] # 제목 저장할 리스트
Kakao_Author = [] # 작가 저장할 리스트
Kakao_Point = [] # 평점 리스트
Kakao_Ranking = [i+1 for i in range(300)]
Links=[]


The chromedriver version (125.0.6422.141) detected in PATH at C:\k-digital\source\python source\myProject\AnalysisProject-Web Novel\AnalysisProject-Web Novel\chromedriver.exe might not be compatible with the detected chrome version (126.0.6478.62); currently, chromedriver 126.0.6478.62 is recommended for chrome 126.*, so it is advised to delete the driver in PATH and retry


In [3]:
# 카카오 페이지는 XPATH로 경로 클릭이 안되서 링크 따온 리스트를 만들고 driver.get()으로 하나씩 열고 필요한 거 크롤링함
for i in range(1, 301, 1):
    time.sleep(0.1)
    element = f'#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div > div.flex.grow.flex-col > div:nth-child(3) > div > div.flex.w-full.grow.flex-col > div > div > div > div:nth-child({i}) > div > a'
    div=driver.find_element(By.CSS_SELECTOR, element).get_attribute('href')
    Links.append(div)

In [4]:
# 링크 하나씩 열고 장르,제목,작가,평점 저장 및 19금 작품의 로그인 필요시 '19금'으로 통일함
for i in range(len(Links)):
    driver.get(Links[i])
    time.sleep(0.3)
    
    try:
        Kakao_Genre.append(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[1]/div').text)
        Kakao_Titles.append(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[1]').text)
        Kakao_Author.append(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[2]').text)
        Kakao_Point.append(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[3]/span').text)
        time.sleep(0.5)
    except NoSuchElementException: # 여기선 네이버랑 다르게 에러가 안떠서 간단하게 함
        driver.back()
        Kakao_Genre.append('19금')
        Kakao_Titles.append('19금')
        Kakao_Author.append('19금')
        Kakao_Point.append('19금')
        time.sleep(0.5)
        continue    

In [5]:
# 데이터프레임으로 만듬
Kakao_Now = pd.DataFrame({'제목': Kakao_Titles, '장르': Kakao_Genre, '작가':Kakao_Author, '평점': Kakao_Point}, index=Kakao_Ranking)
Kakao_Now

,제목,장르,작가,평점
1,마론 후작,웹소설로판,자야,10.0
2,신 내린 명품 디자이너,웹소설현판,신줏단지,9.6
3,흑백무제,웹소설무협,현임,9.8
4,살인귀 황태자를 교육하는 법,웹소설로판,구운빵,9.6
5,천재 궁수의 스트리밍,웹소설판타지,멍멍킴,9.9
...,...,...,...,...
296,황녀가 깽판 침,웹소설로판,박귀리,9.9
297,회귀자의 증권사 생활,웹소설현판,유재무,9.6
298,이혼 후 코인 대박,웹소설현판,어둠의Dark,9.2
299,명가의 역대급 재능천재로 환생했다,웹소설판타지,플라이스,9.7


In [6]:
# 데이터 전처리 필요 확인 단계
Kakao_Now.장르.unique()                       

array(['웹소설로판', '웹소설현판', '웹소설무협', '웹소설판타지', '19금', '웹소설로맨스', '웹소설드라마'],
      dtype=object)

In [7]:
# 데이터 전처리 과정
# '웹소설로판', '웹소설판타지', '웹소설무협', '19금', '웹소설현판', '웹소설로맨스', '웹소설드라마' 등 앞에 웹소설 때고 통일
Kakao_Now.loc[Kakao_Now.장르 == '웹소설로판', '장르'] = '로맨스 판타지'
Kakao_Now.loc[Kakao_Now.장르 == '웹소설판타지', '장르'] = '판타지'
Kakao_Now.loc[Kakao_Now.장르 == '웹소설무협', '장르'] = '무협'
Kakao_Now.loc[Kakao_Now.장르 == '웹소설현판', '장르'] = '현대 판타지'
Kakao_Now.loc[Kakao_Now.장르 == '웹소설로맨스', '장르'] = '로맨스'
Kakao_Now.loc[Kakao_Now.장르 == '웹소설드라마', '장르'] = '드라마'
Kakao_Now

,제목,장르,작가,평점
1,마론 후작,로맨스 판타지,자야,10.0
2,신 내린 명품 디자이너,현대 판타지,신줏단지,9.6
3,흑백무제,무협,현임,9.8
4,살인귀 황태자를 교육하는 법,로맨스 판타지,구운빵,9.6
5,천재 궁수의 스트리밍,판타지,멍멍킴,9.9
...,...,...,...,...
296,황녀가 깽판 침,로맨스 판타지,박귀리,9.9
297,회귀자의 증권사 생활,현대 판타지,유재무,9.6
298,이혼 후 코인 대박,현대 판타지,어둠의Dark,9.2
299,명가의 역대급 재능천재로 환생했다,판타지,플라이스,9.7


In [8]:
# 데이터 전처리 과정으로 평점을 float로 바꾸기 위해 평점에 '19금'을 0.0으로 바꾸고 다시 저장
Kakao_Now.loc[Kakao_Now.평점 == '19금', '평점'] = 0.0
Kakao_Now.평점 = Kakao_Now.평점.astype(float)

In [9]:
all_genres = pd.Series(Kakao_Now['장르'].unique())
print(all_genres) # 로맨스 판타지, 19금 , 무협, 판타지, 로맨스, 현대 판타지, 드라마

for genre in all_genres:
    Kakao_Now[genre] = (Kakao_Now['장르'] == genre)

# 19금은 로그인이 필요해서 평점 데이터가 제대로 수집되지 않음(0.0으로 통일 시킴) 
# 19금은 제외 함
Kakao_Now=Kakao_Now.drop(Kakao_Now[Kakao_Now.제목 == '19금'].index) # 19금 인덱스 삭제
Kakao_Now=Kakao_Now.drop(columns='19금', axis=1)
Kakao_Now.index = [i+1 for i in range(len(Kakao_Now))] # 19금 제외하고 인덱스 번호 다시 
Kakao_Now # 19금을 제외한 270개의 데이터

0    로맨스 판타지
1     현대 판타지
2         무협
3        판타지
4        19금
5        로맨스
6        드라마
dtype: object


,제목,장르,작가,평점,로맨스 판타지,현대 판타지,무협,판타지,로맨스,드라마
1,마론 후작,로맨스 판타지,자야,10.0,True,False,False,False,False,False
2,신 내린 명품 디자이너,현대 판타지,신줏단지,9.6,False,True,False,False,False,False
3,흑백무제,무협,현임,9.8,False,False,True,False,False,False
4,살인귀 황태자를 교육하는 법,로맨스 판타지,구운빵,9.6,True,False,False,False,False,False
5,천재 궁수의 스트리밍,판타지,멍멍킴,9.9,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...
261,황녀가 깽판 침,로맨스 판타지,박귀리,9.9,True,False,False,False,False,False
262,회귀자의 증권사 생활,현대 판타지,유재무,9.6,False,True,False,False,False,False
263,이혼 후 코인 대박,현대 판타지,어둠의Dark,9.2,False,True,False,False,False,False
264,명가의 역대급 재능천재로 환생했다,판타지,플라이스,9.7,False,False,False,True,False,False


In [10]:
# 마지막 확인
Kakao_Now.info() # null값 없음 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265 entries, 1 to 265
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   제목       265 non-null    object 
 1   장르       265 non-null    object 
 2   작가       265 non-null    object 
 3   평점       265 non-null    float64
 4   로맨스 판타지  265 non-null    bool   
 5   현대 판타지   265 non-null    bool   
 6   무협       265 non-null    bool   
 7   판타지      265 non-null    bool   
 8   로맨스      265 non-null    bool   
 9   드라마      265 non-null    bool   
dtypes: bool(6), float64(1), object(3)
memory usage: 11.9+ KB


In [11]:
Kakao_Now.to_csv('./data/Kakao_Now300.txt')

In [14]:
Kakao_Now.loc[(Kakao_Now.장르 == '로맨스') | (Kakao_Now.장르 == '로맨스 판타지')]

,제목,장르,작가,평점,로맨스 판타지,현대 판타지,무협,판타지,로맨스,드라마
1,마론 후작,로맨스 판타지,자야,10.0,True,False,False,False,False,False
4,살인귀 황태자를 교육하는 법,로맨스 판타지,구운빵,9.6,True,False,False,False,False,False
10,멍문세가를 호령하는 아기 고양이입니다,로맨스 판타지,아울킴,9.8,True,False,False,False,False,False
11,시한부를 즐겼을 뿐이었는데,로맨스 판타지,다나,9.9,True,False,False,False,False,False
13,몰락 세가의 시한부 영약,로맨스 판타지,프리드리히,10.0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
251,바람에 흩날리는 꽃비처럼,로맨스,황지원,9.8,False,False,False,False,True,False
252,모용세가의 사라진 딸이 돌아왔다,로맨스 판타지,홍세우,9.8,True,False,False,False,False,False
255,악녀는 마리오네트,로맨스 판타지,한이림,9.9,True,False,False,False,False,False
259,악당을 바르게 키워보겠습니다,로맨스 판타지,한나비,9.8,True,False,False,False,False,False
